In [2]:
import re
import numpy as np
import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow import keras
from keras import layers
from keras.applications import efficientnet
from keras.layers import TextVectorization